# AI/ML Weekly News Reporter

This notebook demonstrates implementing the LangGraph based Weekly News Reporter demo using the Furiosa SDK.

The demo works exactly the same, with only the LLM connection part changed.


In [2]:
import os
import json
from typing import Dict, List, Any, TypedDict, Optional
from datetime import datetime
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph

## Connecting to LLM using OpenAI Compatible Server

Modify the code in the existing demo to connect to a pre-enabled compatibility server instead of the OpenAI API.

In [ ]:

# Initialize API clients
llm = ChatOpenAI(
    model="llama3.1-8b-bf16",
    temperature=0.1,
    base_url = "http://localhost:8000/v1/",
    openai_api_key = "EMPTY",
)


In [4]:
# Test if the API is working well
llm.generate(["Hello, how are you?"], max_tokens=50)

LLMResult(generations=[[ChatGeneration(text="I'm functioning properly, thank you for asking. How can I assist you today?", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="I'm functioning properly, thank you for asking. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 41, 'total_tokens': 59, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bce1fad7-4001-4420-889f-a5afacab1da7-0', usage_metadata={'input_tokens': 41, 'output_tokens': 18, 'total_tokens': 59, 'input_token_details': {}, 'output_token_details': {}}))]], llm_output={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 41, 'total_tokens': 59}, 'model_name': 'llama3.1-8b-bf16'}, run=[RunInfo(run_id=UUID('bce1fad7-4001-4420-889f-a5afacab1da7'

## Define basic data types

In [5]:

class Article(BaseModel):
    """
    Represents a single news article

    Attributes:
        title (str): Article headline
        url (str): Source URL
        content (str): Article content
    """
    title: str
    url: str
    content: str

class Summary(TypedDict):
    """
    Represents a processed article summary

    Attributes:
        title (str): Original article title
        summary (str): Generated summary
        url (str): Source URL for reference
    """
    title: str
    summary: str
    url: str

# This defines what information we can store and pass between nodes later
class GraphState(TypedDict):
    """
    Maintains workflow state between agents

    Attributes:
        articles (Optional[List[Article]]): Found articles
        summaries (Optional[List[Summary]]): Generated summaries
        report (Optional[str]): Final compiled report
    """
    articles: Optional[List[Article]]
    summaries: Optional[List[Summary]]
    report: Optional[str]


## Define Agents

We have modified NewsSearcher agent to reuse cached result for easy testing.

In [ ]:

class NewsSearcher:
    """
    Agent responsible for finding relevant AI/ML news articles
    using the Tavily search API
    """
    
    def search(self) -> List[Article]:
        """
        Performs news search with configured parameters
        
        Returns:
            List[Article]: Collection of found articles
        """

        #response = tavily.search(
            #query="artificial intelligence and machine learning news", 
            #topic="news",
            #time_period="1w",
            #search_depth="advanced",
            #max_results=5
        #)

        response = json.load(open('response.json'))
        
        articles = []
        for result in response['results']:
            articles.append(Article(
                title=result['title'],
                url=result['url'],
                content=result['content']
            ))
        
        return articles

class Summarizer:
    """
    Agent that processes articles and generates accessible summaries
    using gpt-4o-mini
    """
    
    def __init__(self):
        self.system_prompt = """
        You are an AI expert who makes complex topics accessible 
        to general audiences. Summarize this article in 2-3 sentences, focusing on the key points 
        and explaining any technical terms simply.
        """
    
    def summarize(self, article: Article) -> str:
        """
        Generates an accessible summary of a single article
        
        Args:
            article (Article): Article to summarize
            
        Returns:
            str: Generated summary
        """
        response = llm.invoke([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Title: {article.title}\n\nContent: {article.content}")
        ])
        return response.content

class Publisher:
    """
    Agent that compiles summaries into a formatted report 
    and saves it to disk
    """
    
    def create_report(self, summaries: List[Dict]) -> str:
        """
        Creates and saves a formatted markdown report
        
        Args:
            summaries (List[Dict]): Collection of article summaries
            
        Returns:
            str: Generated report content
        """
        prompt = """
        Create a weekly AI/ML news report for the general public. 
        Format it with:
        1. A brief introduction
        2. The main news items with their summaries
        3. Links for further reading
        
        Make it engaging and accessible to non-technical readers.
        """
        
        # Format summaries for the LLM
        summaries_text = "\n\n".join([
            f"Title: {item['title']}\nSummary: {item['summary']}\nSource: {item['url']}"
            for item in summaries
        ])
        
        # Generate report
        response = llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=summaries_text)
        ])
        
        # Add metadata and save
        current_date = '2025-03-14'
        markdown_content = f"""
        Generated on: {current_date}

        {response.content}
        """
        
        #filename = f"ai_news_report_{current_date}.md"
        #with open(filename, 'w') as f:
            #f.write(markdown_content)
        
        return response.content



## Define Workflow Graph using Agents

In [7]:

def search_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node for article search
    
    Args:
        state (Dict[str, Any]): Current workflow state
        
    Returns:
        Dict[str, Any]: Updated state with found articles
    """
    searcher = NewsSearcher()
    state['articles'] = searcher.search() 
    return state

def summarize_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node for article summarization
    
    Args:
        state (Dict[str, Any]): Current workflow state
        
    Returns:
        Dict[str, Any]: Updated state with summaries
    """
    summarizer = Summarizer()
    state['summaries'] = []
    
    for article in state['articles']: # Uses articles from previous node
        summary = summarizer.summarize(article)
        state['summaries'].append({
            'title': article.title,
            'summary': summary,
            'url': article.url
        })
    return state

def publish_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node for report generation
    
    Args:
        state (Dict[str, Any]): Current workflow state
        
    Returns:
        Dict[str, Any]: Updated state with final report
    """
    publisher = Publisher()
    report_content = publisher.create_report(state['summaries'])
    state['report'] = report_content
    return state


def create_workflow() -> StateGraph:
    """
    Constructs and configures the workflow graph
    search -> summarize -> publish

    Returns:
        StateGraph: Compiled workflow ready for execution
    """

    # Create a workflow (graph) initialized with our state schema
    workflow = StateGraph(state_schema=GraphState)

    # Add processing nodes that we will flow between
    workflow.add_node("search", search_node)
    workflow.add_node("summarize", summarize_node)
    workflow.add_node("publish", publish_node)

    # Define the flow with edges
    workflow.add_edge("search", "summarize") # search results flow to summarizer
    workflow.add_edge("summarize", "publish") # summaries flow to publisher

    # Set where to start
    workflow.set_entry_point("search")

    return workflow.compile()


## Execute workflow and get the summary

In [8]:

# Initialize and run workflow
workflow = create_workflow()
final_state = workflow.invoke({
    "articles": None,
    "summaries": None,
    "report": None
})

# Display results
print("\n=== AI/ML Weekly News Report ===\n")
print(final_state['report'])



=== AI/ML Weekly News Report ===

**Weekly AI/ML News Report**

**Introduction**

Welcome to our weekly AI/ML news report, where we bring you the latest updates on the rapidly evolving world of artificial intelligence and machine learning. From breakthroughs in materials science to innovative applications in healthcare and finance, we've got you covered. This week, we're highlighting the latest developments in AI-powered hotel management, the creation of ultra-thin metals, and the transformation of the private equity industry.

**Main News Items**

1. **SiteMinder Appoints New Chiefs of Data, Tech to Unlock the Power of AI, Machine Learning for Hotels**

SiteMinder, a leading hotel management platform, has appointed two new leaders to drive the adoption of AI and machine learning in the hospitality industry. Mike Rogers, co-founder of SiteMinder, will lead the company's data strategy as Chief Data Officer, while another executive will oversee the development of a "Smart Platform" that